<a href="https://colab.research.google.com/github/imabari/ehime-covid-bot/blob/main/ehime_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# インストール

In [1]:
!pip install pdfminer.six

     |████████████████████████████████| 5.6MB 11.5MB/s 
     |████████████████████████████████| 2.6MB 58.5MB/s 


# プログラム

## 設定

In [2]:
import datetime
import pathlib
import re
import unicodedata
from urllib.parse import urljoin

In [3]:
from pdfminer.high_level import extract_text
import requests
import tweepy
from bs4 import BeautifulSoup

In [4]:
# 愛媛県の新型コロナウイルス感染症に関する情報
url = "https://www.pref.ehime.jp/h25500/kansen/covid19.html"

## 関数

In [5]:
# スクレイピング
def fetch_soup(url, parser="html.parser"):

    r = requests.get(url)
    r.raise_for_status()

    soup = BeautifulSoup(r.content, parser)

    return soup

In [6]:
# PDFファイルからテキスト抽出
def fetch_text(url):

    r = requests.get(url)

    p = pathlib.Path(pathlib.PurePath(url).name)

    with p.open(mode="wb") as fw:
        fw.write(r.content)

    text = extract_text(p)

    return unicodedata.normalize("NFKC", text)

## スクレイピング

In [7]:
soup = fetch_soup(url)

In [8]:
# 記事のリンクを抽出
tags = soup.select("div#tmp_contents > ul > li > a")

## 新型コロナウイルスの感染の確認等について

In [9]:
# 最新記事は上から4番目
tag1 = tags[3]

In [10]:
# タイトル確認
title1 = tag1.get_text(strip=True)
print(title1)

（1月23日公表分）新型コロナウイルスの感染の確認等について（125・173・182・198-2・258・271・282・286・292～294事例目及び新規2事例）（PDF：155KB）


In [11]:
# PDFのURLを作成
link1 = urljoin(url, tag1.get("href"))

In [12]:
# PDFファイルをダウンロード
text1 = fetch_text(link1)

In [13]:
# テキスト確認
print(text1)

910 人目 

~
931 人目 

 

10 歳未満:1名 
10 代    :1名 
20 代    :2名 
30 代    :4名 
50 代    :4名 
60 代    :1名 
70 代    :1名 
80 代    :3名 
90 代    :5名 

あり 

なし 
 

 
 

8名 
 

愛媛県保健福祉部健康衛生局 
健康増進課(内線 2402) 

令和3年1月 23 日 

 

 
 

 

新型コロナウイルスの感染の確認について 

昨日(1月 22 日)、県内で新型コロナウイルスの陽性者が 22 名確認されました。 
積極的疫学調査及び健康観察等を実施してまいります。 

 
○既存・新規人数 
・濃厚接触者及び接触者の検査により判明した陽性者数:20 名 
・上記以外の検査により新たに判明した陽性者数      :2名 
 
○陽性者の概要 
陽性者 

居住地 

年代 

性別 

職業 

男性:10 名 
 
女性:12 名 

松山市    :12 名 
今治市    :1名 
大洲市    :1名 
四国中央市:5名 
西予市    :2名 
東温市    :1名 

公務員    :1名 
会社員    :3名 
福祉関係  :3名 
学校関係  :5名 
無  職    :10 名 

 
○症状の有無                               ○感染経路等 

       14 名 ※全て軽症 

  家庭内 

  同僚・職場内 

  医療機関・社会福祉施設等内 
  生活上の接触 
  調査中 
 

 
※現段階の調査で感染経路の可能性が推測される 

:5名 

:4名 

:10 名 
:1名 
:2名 
 

 

   

 

 

 
 
○クラスターの状況(陽性者の概要等は上記に含まれる) 

場面等 

事例 

125 事例目 

新たな陽性者 
医療機関以外 
の者2名 

173 事例目  警察署1名 

198-2 事例目  入所者1名 

258 事例目 

職  員1名 
入所者1名 

271 事例目  入所者5名 
                         

陽性者の状況 

医療機関クラスター2 
累計陽性者 24 名(職員 6

# 出力

### 全体

In [14]:
m1 = re.search("昨日\((.+)\)、県内で新型コロナウイルスの陽性者が(.+)名確認されました。", text1)
s1 = m1.group(0).replace(" ", "")
print(s1)

昨日(1月22日)、県内で新型コロナウイルスの陽性者が22名確認されました。


### 既存事例

In [15]:
m2 = re.search("濃厚接触者及び接触者の検査により判明した陽性者数 *:(.+)名", text1)
s2 = m2.group(0).replace(" ", "")
print(s2)

濃厚接触者及び接触者の検査により判明した陽性者数:20名


### 新規事例

In [16]:
m3 = re.search("上記以外の検査により新たに判明した陽性者数 *:(.+)名", text1)
s3 = m3.group(0).replace(" ", "")
print(s3)

上記以外の検査により新たに判明した陽性者数:2名


### 文章結合

In [17]:
print(f"{s1}\n{s2}\n{s3}")

昨日(1月22日)、県内で新型コロナウイルスの陽性者が22名確認されました。
濃厚接触者及び接触者の検査により判明した陽性者数:20名
上記以外の検査により新たに判明した陽性者数:2名


# 愛媛県内の状況

In [18]:
# 記事は上から3番目
tag2 = tags[2]

In [19]:
# タイトル確認
title2 = tag2.get_text(strip=True)
print(title2)

（1月23日9時現在）愛媛県内の状況（PDF：315KB）


In [20]:
# PDFのURLを作成
link2 = urljoin(url, tag2.get("href"))

In [21]:
# PDFファイルをダウンロード
text2 = fetch_text(link2)

In [22]:
# テキスト確認
print(text2)

愛 媛 県 内 の 状 況

【 R3.1.23 9時現在 】

事 例

公表日 検査数

陰 性

陽 性

関係者
調査

PCR
検査

健康
観察

5,081

4,538

543

<封じ込め事例>

対処事例1:194事例
※193、252、262:欠番

松山市保健所:3事例
(192、196、201事例目)

今治保健所:1事例
(263事例目)

宇和島保健所:1事例
(197事例目)

<囲い込み事例>

対処事例2
(136、156-2、160、163、
169、186、189、194、
198-1、199、200、
202~209、211、
213、214、216~218、
221~223、225~228、
230、231、232、233、
234、236~242、244、
246、248~251、254、
255、256、257、259、
265~268、270、
273事例目)
 125事例目
【会食3、医療機関2】
198-2事例目
【高齢者施設4】
178事例目
(松山市保健所)
215事例目
(八幡浜保健所)
松山市保健所:4事例
(156-1、253、260、
280事例目)

四国中央保健所:1事例
(235事例目)

西条保健所:1事例
(275事例目)

今治保健所:1事例
(272事例目)

八幡浜保健所:2事例
(276、277事例目)

20

1

20

15

0

19

12/25

1/16

1/7

1/11

(2)
388
(17)
59
(1)
33
(52)
155

153

2

27
(4)
6

26

359
(16)
52
(1)
30
(52)
151

113

1

25
(4)
5

24

●

●

●

●

●

●

●

●

●

●

●

●

●

5

1

1

(2)
29
(1)
7

40

3

4

1

2

1

2

●

●

●

●

●

●

●

●

●

●

●

●

●

●

●

●

●

〇

〇

〇

〇

〇

〇

〇

〇

〇

1,631

1,442

189

●

●

○

愛 媛 県 内 の 状 況

【 R3.1.23 9時現在 】

<調査中事例>

対処事例3
(220

In [23]:
m4 = re.search("^新規事例\n(\d+)事例合計", text2, re.MULTILINE | re.DOTALL)
s4 = m4.group(0).replace(" ", "")
print(s4)

新規事例
2事例合計


# 抽出

In [24]:
result = {}

In [25]:
result["日付"] = m1.group(1).replace(" ", "")
result["全体人数"] = m1.group(2).replace(" ", "")
result["既存人数"] = m2.group(1).replace(" ", "")
result["新規人数"] = m3.group(1).replace(" ", "")
result["新規事例"] = m4.group(1).replace(" ", "")

In [26]:
print(result)

{'日付': '1月22日', '全体人数': '22', '既存人数': '20', '新規人数': '2', '新規事例': '2'}


In [27]:
twit = f'{result["日付"]}\n全体{result["全体人数"]}名\n既存事例{result["既存人数"]}名\n新規{result["新規事例"]}事例{result["新規人数"]}名'
print(twit)

1月22日
全体22名
既存事例20名
新規2事例2名
